In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
model = UserKNNCFRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2559 (20.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    topK = trial.suggest_int("topK", 1e2, 1e4)
    shrink = trial.suggest_int("shrink", 0, 100)
    similarity = trial.suggest_categorical("similarity", ["asymmetric"])
    #similarity = trial.suggest_categorical("similarity", ["cosine", "jaccard", "asymmetric", "dice", "tversky"])
    normalize = trial.suggest_categorical("normalize", [True, False])
    feature_weighting = trial.suggest_categorical("feature_weighting", ["none", "BM25", "TF-IDF"])
    recommender = model(URM_train)
    if feature_weighting == "tversky": 
        tversky_alpha = trial.suggest_float("tversky_alpha", 0, 2)
        tversky_beta = trial.suggest_float("tversky_beta", 0, 2)
        normalize = True
        tversky_params = [tversky_alpha, tversky_beta, normalize]
        recommender.fit(**trial.params, **tversky_params)
    elif similarity == "asymmetric":
        asymmetric_alpha = trial.suggest_float("asymmetric_alpha", 0, 2)
        normalize = True
        params = [asymmetric_alpha, normalize]
        recommender.fit(**trial.params, **params)
    else :
        recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [4]:
best_params = {
    'topK': 529,
    'shrink': 45,
    'similarity': 'asymmetric',
    'normalize': True,
    'feature_weighting': 'TF-IDF',
    'asymmetric_alpha': 0.0,
}
study_name = "userknn-asymmetric-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-12-04 22:36:17,283] A new study created in RDB with name: userknn-asymmetric-study
[W 2023-12-04 22:36:17,380] Trial 0 failed with parameters: {'topK': 529, 'shrink': 45, 'normalize': True, 'feature_weighting': 'TF-IDF', 'asymmetric_alpha': 0.0} because of the following error: TypeError('fit() argument after ** must be a mapping, not list').
Traceback (most recent call last):
  File "/home/alessandro/RecSys-2023-polimi/venv/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_6147/3272804480.py", line 20, in objective
    recommender.fit(**trial.params, **params)
TypeError: fit() argument after ** must be a mapping, not list
[W 2023-12-04 22:36:17,381] Trial 0 failed with value None.


UserKNNCFRecommender: URM Detected 457 ( 3.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 322 ( 1.4%) items with no interactions.


TypeError: fit() argument after ** must be a mapping, not list

In [5]:
study.best_params

{'topK': 529,
 'shrink': 45,
 'similarity': 'asymmetric',
 'normalize': True,
 'feature_weighting': 'TF-IDF'}

In [7]:
final = model(URM_train_validation)
final.fit(**study.best_params)

SLIMElasticNetRecommender: URM Detected 232 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 103 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 17389 (78.3%) in 5.00 min. Items per second: 57.96
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 6.45 min. Items per second: 57.42


In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2176 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10462 (100.0%) in 6.56 sec. Users per second: 1596


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.096683                 0.167108  0.140668  0.049947    0.086206   
 
             MRR      NDCG      F1  HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                     ...                 
 10      0.27832  0.154074  0.1146  0.527337      0.368245  ...      0.827821   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.436541    0.827821       0.054661        10.16803   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.997643             0.156901              0.783344   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff